# Word embeddings

Este tutorial contiene una introducción a las embeddings de palabras. Entrenaréis vuestros propios embeddings de palabras utilizando un modelo simple de Keras para una tarea de clasificación de sentimiento, y luego podrá ser visualizados los resultados en el [Proyector de Embeddings](http://projector.tensorflow.org).

## Representación del texto como números

Los modelos de aprendizaje automático toman vectores (matrices de números) como entrada. Cuando se trabaja con texto, lo primero que hay que hacer es idear una estrategia para convertir las cadenas en números (o «vectorizar» el texto) antes de introducirlo en el modelo. En esta sección, veremos tres estrategias para hacerlo.

### 1. Codificaciones de una sola vez

Una de las aproximaciones más simples es codificar «una sola vez» cada palabra de su vocabulario. Piensa en la frase «El gato se sentó en la alfombra». El vocabulario (o las palabras únicas) de esta frase es (gato, colchoneta, sobre, se sentó, el). Para representar cada palabra, se crea un vector cero con una longitud igual a la del vocabulario y, a continuación, se coloca un uno en el índice correspondiente a la palabra.

Para crear un vector que contenga la codificación de la frase, se trata unicamente de concatenar los 1s.

**Punto clave**: Un vector codificado con una sola palabra es disperso (es decir, la mayoría de los índices son cero). Imagina que tienes 10.000 palabras en el vocabulario. Para codificar cada palabra de una sola vez, tendría que crear un vector en el que el 99,99% de los elementos fueran cero

### 2. Codificar cada palabra con un número único

Una segunda posibilidad es codificar cada palabra con un **número único**. Continuando con el ejemplo anterior, podrías asignar 1 a «gato», 2 a «alfombrilla», y así sucesivamente. Así, podrías codificar la frase «El gato se sentó en la alfombra» como un vector denso del tipo [5, 1, 4, 3, 5, 2]. Este método es eficaz. En lugar de un vector disperso, ahora tenemos uno denso (en el que todos los elementos están llenos).

Sin embargo, este método tiene dos inconvenientes:

* La codificación entera es arbitraria (no captura ninguna relación entre palabras).

* Una codificación entera puede ser difícil de interpretar para un modelo. Un clasificador lineal, por ejemplo, aprende un único peso para cada característica. Dado que no existe ninguna relación entre la similitud de dos palabras y la similitud de sus codificaciones, esta combinación de características y pesos no tiene sentido.

### 3. Word embeddings

Los embeddings de palabras nos permiten utilizar **una representación densa y eficaz** en la que palabras similares tienen una codificación similar. Y lo que es más importante, no es necesario especificar esta codificación a mano. Un embedding es un vector denso de valores de coma flotante (la longitud del vector es un parámetro que usted especifica). En lugar de especificar manualmente los valores del embedding, se trata de parámetros entrenables (pesos aprendidos por el modelo durante el entrenamiento, del mismo modo que un modelo aprende pesos para una capa densa). Es habitual ver embeddings de palabras de 8 dimensiones (para conjuntos de datos pequeños), hasta 1024 dimensiones cuando se trabaja con conjuntos de datos grandes. Un embedding de mayor dimensión puede captar relaciones más precisas entre las palabras, pero requiere más datos para su aprendizaje.

![embedding](img/mbedding.webp)


## Configuración

Usaremos Keras para trabajar con nuestros vectores de palabras.

In [1]:
import io
import os
import re
import string
import tensorflow as tf

2024-07-24 09:14:13.367544: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-24 09:14:13.376326: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 09:14:13.385828: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 09:14:13.388605: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-24 09:14:13.396364: I tensorflow/core/platform/cpu_feature_guar

### Descargar el conjunto de datos IMDb
Durante el tutorial se utilizará el [Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/). Entrenarás un modelo clasificador de sentimiento en este conjunto de datos y en el proceso aprenderás incrustaciones desde cero.

Echa un vistazo al directorio `train/`. Tiene las carpetas `pos` y `neg` con críticas de películas etiquetadas como positivas y negativas respectivamente. Utilizarás las críticas de las carpetas `pos` y `neg` para entrenar un modelo de clasificación binario.

In [2]:
dataset_dir = os.path.join(os.getcwd(), "aclImdb", "train")
dataset_dir

'/home/iraitz/TheBridge/Otros/TheBridge_DSPT_ML/Lenguaje Natural/3-Embeddings/aclImdb/train'

A continuación, crea un `tf.data.Dataset` usando `tf.keras.preprocessing.text_dataset_from_directory`. Puedes leer más sobre el uso de esta utilidad en este [tutorial de clasificación de texto](https://www.tensorflow.org/tutorials/keras/text_classification). 

Utilice el directorio `train` para crear conjuntos de datos de entrenamiento y validación con una división del 20% para la validación.

In [3]:
batch_size = 1024
seed = 123

train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    dataset_dir,
    batch_size = batch_size,
    validation_split = 0.2,
    subset = 'training',
    seed = seed
)

val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    dataset_dir,
    batch_size = batch_size,
    validation_split = 0.2,
    subset = 'validation',
    seed = seed
)

Found 75000 files belonging to 3 classes.
Using 60000 files for training.


I0000 00:00:1721805257.941399  173887 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721805257.962479  173887 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721805257.962626  173887 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721805257.963756  173887 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Found 75000 files belonging to 3 classes.
Using 15000 files for validation.


Observa algunas críticas de películas y sus etiquetas `(1: positivo, 0: negativo)` del conjunto de datos de entrenamiento.

In [4]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(5): 
    print(label_batch[i].numpy(), text_batch.numpy()[i])

1 b"Ask yourself where she got the gun? Remember what she was taught about the mark's mindset when the con is over? The gun had blanks and it was provided to her from the very beginning.<br /><br />When the patient comes back at the end she was SUPPOSED to see him drive away in the red convertible and lead her to the gang splitting up her 80 thousand.<br /><br />The patient was in on the con from the beginning.<br /><br />Mantegna does not die in the end - the gun had blanks.<br /><br />There - enough spoilers for you there? This is why people are giving it such high ratings. It's extremely original because of the hidden ending and how it cons MOST of the audience."
2 b"For some reason, people seem to have a problem differentiating this movie from Trail of the Pink Panther.<br /><br />At any rate, this work does nothing but serve to remind us how sad the world is without Peter Sellers in it.<br /><br />They brought back the same old favorites from Trail (Dreyfus, Cato, Litton, etc.), b

2024-07-24 09:14:19.657944: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Usando la capa Embedding

Keras facilita el uso de incrustaciones de palabras. Echa un vistazo a la capa [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding).

La capa Embedding puede ser entendida como una tabla de búsqueda que mapea desde índices enteros (que representan palabras específicas) a vectores densos (sus incrustaciones). La dimensionalidad (o anchura) de la incrustación es un parámetro con el que puede experimentar para ver qué funciona bien para su problema, del mismo modo que experimentaría con el número de neuronas en una capa densa.

In [5]:
embedding_layer = tf.keras.layers.Embedding(1000, 5)

Cuando se crea una capa de embedding, los pesos del embedding se inicializan aleatoriamente (como cualquier otra capa). Durante el entrenamiento, se ajustan gradualmente mediante retropropagación. Una vez entrenadas, las incrustaciones de palabras aprendidas codificarán aproximadamente las similitudes entre palabras (ya que fueron aprendidas para el problema específico en el que se entrena el modelo).

Si pasa un número entero a una capa de embedding, el resultado sustituye cada número entero por el vector de la tabla de embedding:

In [6]:
result = embedding_layer(tf.constant([0,1,2,3,4,999]))
result.numpy()

array([[-0.04623629, -0.04370539,  0.03233461, -0.00733647, -0.036759  ],
       [ 0.00241052, -0.02127037, -0.01343727,  0.03349702,  0.01657147],
       [ 0.01297109,  0.00483005,  0.0022527 ,  0.03880982,  0.01060029],
       [-0.04881912, -0.01761485, -0.03210603,  0.00977444,  0.01555166],
       [-0.0046904 , -0.03185989, -0.04879421, -0.02360058,  0.03896526],
       [ 0.02033808,  0.02909926,  0.03333262,  0.03596048,  0.01132948]],
      dtype=float32)

In [7]:
print(embedding_layer.embeddings.shape)
embedding_layer.embeddings

(1000, 5)


<KerasVariable shape=(1000, 5), dtype=float32, path=embedding/embeddings>

Para problemas de texto o secuencias, la capa de embedding toma un tensor 2D de enteros, de forma `(muestras, longitud_secuencia)`, donde cada entrada es una secuencia de enteros. Puede incrustar secuencias de longitudes variables. Podría introducir en la capa de embedding anterior lotes con formas `(32, 10)` (lote de 32 secuencias de longitud 10) o `(64, 15)` (lote de 64 secuencias de longitud 15).

El tensor devuelto tiene un eje más que el de entrada, los vectores de embedding se alinean a lo largo del nuevo último eje. Pásele un lote de entrada `(2, 3)` y la salida será `(2, 3, N)`.

In [8]:
result = embedding_layer(tf.constant([[1, 2, 999],
                                      [3, 4, 5]]))
print(result.shape)
result.numpy()

(2, 3, 5)


array([[[ 0.00241052, -0.02127037, -0.01343727,  0.03349702,
          0.01657147],
        [ 0.01297109,  0.00483005,  0.0022527 ,  0.03880982,
          0.01060029],
        [ 0.02033808,  0.02909926,  0.03333262,  0.03596048,
          0.01132948]],

       [[-0.04881912, -0.01761485, -0.03210603,  0.00977444,
          0.01555166],
        [-0.0046904 , -0.03185989, -0.04879421, -0.02360058,
          0.03896526],
        [ 0.04388979,  0.03584692,  0.04688055,  0.03256333,
         -0.01453091]]], dtype=float32)

Cuando se le da un lote de secuencias como entrada, una capa de embedding devuelve un tensor 3D de coma flotante, de forma `(muestras, longitud_de_secuencia, dimensionalidad_de_embedding)`. Para convertir de esta secuencia de longitud variable a una representación fija hay una variedad de enfoques estándar. Se puede utilizar una capa RNN, Attention o pooling antes de pasarla a una capa Dense. Este tutorial utiliza pooling porque es el más sencillo.

## Procesado de texto

A continuación, definiremos los pasos de preprocesamiento del conjunto de datos necesarios para su modelo de clasificación de sentimientos. Inicialice una capa TextVectorization con los parámetros deseados para vectorizar las críticas de películas.

In [9]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')

vocab_size = 10000
sequence_length = 100

vectorize_layer = tf.keras.layers.TextVectorization(
    standardize = custom_standardization,
    max_tokens = vocab_size,
    output_mode = 'int',
    output_sequence_length = sequence_length
)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

2024-07-24 09:14:28.898866: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Crear un modelo de clasificación

Utilizaremos la [Keras Sequential API](https://www.tensorflow.org/guide/keras/sequential_model) para definir el modelo de clasificación de sentimiento. En este caso es un modelo del estilo «Continuous bag of words».
* La capa [`TextVectorization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization) transforma cadenas en índices de vocabulario. Ya has inicializado `vectorize_layer` como una capa de TextVectorization y construido su vocabulario llamando a `adapt` en `text_ds`. Ahora vectorize_layer se puede utilizar como la primera capa de su modelo de clasificación de extremo a extremo, la alimentación de cadenas transformadas en la capa de incrustación.
* La capa [`Embedding`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) toma el vocabulario codificado con enteros y busca el vector de incrustación para cada índice de palabra. Estos vectores se aprenden a medida que el modelo se entrena. Los vectores añaden una dimensión a la matriz de salida. Las dimensiones resultantes son: `(lote, secuencia, incrustación)`.

* La capa [`GlobalAveragePooling1D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalAveragePooling1D) devuelve un vector de salida de longitud fija para cada ejemplo promediando sobre la dimensión de secuencia. Esto permite al modelo manejar entradas de longitud variable de la forma más sencilla posible.

* El vector de salida de longitud fija se canaliza a través de una capa totalmente conectada ([`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)) con 16 unidades ocultas.

* La última capa está densamente conectada con un único nodo de salida. 

Atención: Este modelo no utiliza enmascaramiento, por lo que el relleno cero se utiliza como parte de la entrada y, por tanto, la longitud del relleno puede afectar a la salida.  Para solucionarlo, consulta la [guía de enmascaramiento y relleno](https://www.tensorflow.org/guide/keras/masking_and_padding).

In [10]:
embedding_dim = 16
vocab_size = 10000

model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(vocab_size, embedding_dim, name='embedding'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

## Compilamos y entrenamos el modelo

Utilizarás [TensorBoard](https://www.tensorflow.org/tensorboard) para visualizar métricas incluyendo pérdida y precisión. Crearemos un `tf.keras.callbacks.TensorBoard`.

In [11]:
tbcallback = tf.keras.callbacks.TensorBoard(
    log_dir='logs',
    histogram_freq=0,
    write_graph=True,
    write_images=False,
    write_steps_per_second=False,
    update_freq='epoch',
    profile_batch=0,
    embeddings_freq=0,
    embeddings_metadata=None
)

model.compile(optimizer='adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

Compila y entrena usando el optimizador `Adam` y la pérdida `BinaryCrossentropy`. 

In [15]:
model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 10,
    callbacks=[tbcallback]
)

Epoch 1/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.1695 - loss: -21891.4609 - val_accuracy: 0.1690 - val_loss: -23107.2734
Epoch 2/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.1692 - loss: -23934.3145 - val_accuracy: 0.1690 - val_loss: -25161.6426
Epoch 3/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.1689 - loss: -25925.4883 - val_accuracy: 0.1690 - val_loss: -27315.2090
Epoch 4/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.1681 - loss: -28450.2324 - val_accuracy: 0.1690 - val_loss: -29577.2695
Epoch 5/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.1675 - loss: -30538.3145 - val_accuracy: 0.1690 - val_loss: -31934.5352
Epoch 6/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.1672 - loss: -33275.8516 - val_accuracy: 0.1690 - val_loss: -34406.0898
Epoch 7/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.1692 - loss: -35498.2461 - val_accuracy: 0.1690 - val_loss: -36981.7930
Epoch 8/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 

In [13]:
!tensorboard --logdir=logs

2024-07-24 09:15:06.681335: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-24 09:15:06.689904: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 09:15:06.699444: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 09:15:06.702257: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-24 09:15:06.709865: I tensorflow/core/platform/cpu_feature_guar

Se ve que en nuestro caso el resultado no es muy bueno, toca jugar con la arquitectura para ver mejores resultados.

Puede consultar el resumen del modelo para obtener más información sobre cada capa del modelo.

In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 16)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 16)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 480,869 (1.83 MB)

 Trainable params: 160,289 (626.13 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 320,580 (1.22 MB)

## Recupera los embeddings de palabras entrenadas y guardalas

A continuación, recupere los embeddings de palabras aprendidas durante el entrenamiento. Los embeddings son pesos de la capa Embedding en el modelo. La matriz de pesos tiene la forma `(vocab_size, embedding_dimension)`.

In [17]:
model.get_layer('embedding').get_weights()

[array([[-144.44925   , -142.90196   ,  145.35532   , ..., -144.58495   ,
          143.64536   , -140.99751   ],
        [-175.28365   , -173.52979   ,  175.6521    , ..., -175.8469    ,
          174.50255   , -171.24246   ],
        [-166.29767   , -164.64468   ,  166.68562   , ..., -166.78928   ,
          165.55664   , -162.46303   ],
        ...,
        [  -1.4809538 ,   -1.4849576 ,    1.5274029 , ...,   -1.5311236 ,
            1.527871  ,   -1.4855096 ],
        [  -1.3023555 ,   -1.2495835 ,    1.2694421 , ...,   -1.317147  ,
            1.3137275 ,   -1.2370903 ],
        [  -0.6727082 ,   -0.61914897,    0.66247225, ...,   -0.61728567,
            0.63086647,   -0.6857476 ]], dtype=float32)]

In [18]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [19]:
print(len(vocab))
print(vocab[:10])

10000
['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it']


In [20]:
print(weights.shape)
print(weights[:2])

(10000, 16)
[[-144.44925 -142.90196  145.35532  142.65868  142.36775 -142.2914
   142.062   -144.477    144.83801 -143.59645 -143.00455 -146.32506
  -143.02974 -144.58495  143.64536 -140.99751]
 [-175.28365 -173.52979  175.6521   173.12965  172.70372 -172.84514
   172.24977 -175.66295  175.90012 -174.243   -173.44789 -177.46103
  -173.57501 -175.8469   174.50255 -171.24246]]


Guardaremos los resultados para poder emplearlos como fuente en el [Embedding Projector](http://projector.tensorflow.org).

In [21]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

## Visualiza los Embeddings

Para visualizar los embeddings, cárgalos en el proyector de embeddings.

Abrir el [Proyector integrado](http://projector.tensorflow.org/) (esto también se puede ejecutar en una instancia local de TensorBoard).

* Haz clic en "Cargar datos".

* Sube los dos archivos que creaste anteriormente: `vecs.tsv` y `meta.tsv`.

Ahora se mostrarán los embeddings que ha entrenado. Puedes buscar palabras para encontrar sus vecinos más cercanos. Por ejemplo, intenta buscar "hermosa". Es posible que vea vecinos como "maravillosos".


## Tutoriales de ayuda

Este tutorial te ha mostrado cómo entrenar y visualizar embeddings de palabras desde cero en un pequeño conjunto de datos.

* Para entrenar embeddings de palabras usando el algoritmo Word2Vec, prueba el tutorial [Word2Vec](https://www.tensorflow.org/tutorials/text/word2vec).

* Para obtener más información sobre el procesamiento de texto avanzado, lee el [Modelo Transformer para la comprensión del lenguaje](https://www.tensorflow.org/tutorials/text/transformer).

* Echad un ojo a [Llama Index](https://docs.llamaindex.ai/en/stable/module_guides/models/embeddings/)